In [ ]:
import datetime as dt
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from flask import Flask, jsonify


#################################################
# Database Setup
#################################################
engine = create_engine("sqlite:///hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save reference to the table
Measurement = Base.classes.measurement
Station = Base.classes.station

#################################################
# Session link from Python to SQLLite DB
#################################################
session = Session(engine)


#################################################
# Flask Setup
#################################################
app = Flask(__name__)


#################################################
# Flask Routes
#################################################

@app.route("/")
def welcome():
    """List all available api routes for Hawaii Climate."""
    return (
        f"Hawaii Climate API<br/>"
        f"Available Routes:<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/temp/start/end"
    )

@app.route("/api/v1.0/precipitation")
def prec():
    # Return prec data from last year
    prev_year = dt.date(2017,8,23) - dt.timedelta(days=365)
    prec = session.query(Measurement.date, Measurement.prcp).\
        filter(Measurement.date >= prev_year).all()
    session.close()
    precipitat = {date: prcp for date, prcp in prec}
    return jsonify(precipitat)

@app.route("/api/v1.0/stations")
def stations():
    results = session.query(Station.station).all
    session.close()
    stations=list(np.ravel(results))
    return jsonify(stations)

if __name__ == '__main__':
    app.run(debug=True)
